In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import seaborn as sns
from scipy import stats
import analysis_perDRG
import scikit_posthocs as sp
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
def statistic_all(feature):
    group_a_data = getattr(sham_il, feature)
    group_b_data = getattr(sham_cl, feature)    
    group_c_data = getattr(sni_il, feature)
    group_d_data = getattr(sni_cl, feature)
    
    # Test for normal distribution
    statistic, p_value_a = stats.shapiro(group_a_data)
    statistic, p_value_b = stats.shapiro(group_b_data)
    statistic, p_value_c = stats.shapiro(group_c_data)
    statistic, p_value_d = stats.shapiro(group_d_data)
    
    if (p_value_b < 0.05) or (p_value_a < 0.05) or (p_value_c < 0.05) or (p_value_d < 0.05):
        # data is not normally distrubuted (<0.05)
        # non-parametric version of ANOVA
        statistic, p_value = stats.kruskal(group_a_data, group_b_data, group_c_data, group_d_data, nan_policy='omit')
        posthoc = 'Mann Whitney U'
        test = 'Kruskal-Wallis H-test'
        P_value = p_value.round(4)
        Statistic = statistic.round(4)
        #perform posthoc mannwhitney-u
        stat = sp.posthoc_mannwhitney([pd.Series(group_a_data).dropna(), pd.Series(group_b_data).dropna(), 
                                       pd.Series(group_c_data).dropna(), pd.Series(group_d_data).dropna()],
                                       p_adjust='Bonferroni')
        
    else:
        # data is normally distributed
        # test for equal variance
        statistic, p_value = stats.bartlett(pd.Series(group_a_data).dropna(), pd.Series(group_b_data).dropna(), pd.Series(group_c_data).dropna(), pd.Series(group_d_data).dropna())
        if p_value < 0.05:
            # no equal variance
            equal_variance = False
            # tests the equality of k independent means in the face of heterogeneity of variance
            p_value = stats.alexandergovern(group_a_data, group_b_data, group_c_data, group_d_data)
            #print('Welch’s t-test: p-value='+"{:.5f}".format(p_value))
            posthoc = 'Welch’s t-test'
            test = 'Alexander-Govern'
            P_value = 42
            Statistic = 42
 
        else: 
            #data is normally distributed and have equal variances
            equal_variance = True
            statistic, p_value = stats.f_oneway(pd.Series(group_a_data).dropna(), pd.Series(group_b_data).dropna(), pd.Series(group_c_data).dropna(), pd.Series(group_d_data).dropna())
            #print('one-way ANOVA: p-value='+"{:.5f}".format(p_value))
            test = 'one-way ANOVA'
            posthoc = 't-test'
            P_value = p_value.round(4)
            Statistic = statistic.round(4)            
        stat = sp.posthoc_ttest([pd.Series(group_a_data).dropna(), pd.Series(group_b_data).dropna(), 
                                 pd.Series(group_c_data).dropna(), pd.Series(group_d_data).dropna()], 
                                equal_var=equal_variance, p_adjust='Bonferroni') 
        #, p_adjust = 'bonferroni'
    
    stat.columns = ['Sham_IL', 'Sham_CL', 'SNI_IL', 'SNI_CL']
    stat.index = ['Sham_IL', 'Sham_CL', 'SNI_IL', 'SNI_CL']
    return test, Statistic, P_value, posthoc, stat.round(4)

In [3]:
def make_table(features, names):
    statistics = pd.DataFrame(columns=['Metric', 'Test', 'statistic', 'p-value', 'Post-hoc', 'Sham IL vs Sham CL', 'Sham IL vs SNI IL', 'SNI IL vs SNI CL', 'Sham CL vs SNI CL'])
    for feature, name, i in zip(features, names, range(len(features))):
        test, statistic, p_value, posthoc, p_values = statistic_all(feature)
        statistics.loc[i] = [name, test, statistic, p_value, posthoc, p_values['Sham_IL']['Sham_CL'], p_values['Sham_IL']['SNI_IL'], p_values['SNI_IL']['SNI_CL'], p_values['Sham_CL']['SNI_CL']]
    return statistics                                                              

In [4]:
features = ['gfap_intensities', 'gs_intensities', 'gfap_integrated_density', 'gs_integrated_density',  
            'neurons_per_tissue', 'small_neurons_percentage', 'bigger_neurons_percentage',
            'gfap_area_per_tissue', 'gfap_area_per_neurons', 'ring_ratios_gfap', 'small_neurons_gfap_ring', 'bigger_neurons_gfap_ring',
            'gs_area_per_tissue', 'gs_area_per_neurons', 'ring_ratios_gs', 'small_neurons_gs_ring', 'bigger_neurons_gs_ring',
            'ring_area_per_tissue', 'ring_area_per_neurons', 'ring_ratios_ring', 'gs_overlaps', 'gfap_overlaps']
names = ['GFAP intensities', 'GS intensities', 'GFAP integrated density', 'GS integrated density', 
         'Neurons/mm^2', 'Small neurons (%)', 'Medium-sized and large neurons (%)',
         'GFAP area/ tissue area (%)', 'GFAP area/ neuronal area (%)', 'Neurons in proximity to GFAP+ glial cell (%)', 'Small neurons in proximity to GFAP+ glial cell (%)', 'Medium-sized and large neurons in proximity to GFAP+ glial cell (%)',
         'GS area/ tissue area (%)', 'GS area/ neuronal area (%)', 'Neurons in proximity to GS+ glial cell (%)', 'Small neurons in proximity to GS+ glial cell (%)', 'Medium-sized and large neurons in proximity to GS+ glial cell (%)',
         'Ring area/ tissue area (%)', 'Ring area/ neuronal area (%)', 'Neurons in proximity to glial cell (%)', 'GS+ glial cells expressing GFAP (%)', 'GFAP+ glial cells expressing GS (%)']

In [5]:
name = 'd7perDRG_'

with open('D7_SNI_area.json') as f:
    results_sni = json.load(f)
with open('D7_Sham_area.json') as f:
    results_sham = json.load(f)
    
sham_il = analysis_perDRG.GroupData(results_sham, 'IL', number_of_rats=6)
sham_cl = analysis_perDRG.GroupData(results_sham, 'CL', number_of_rats=6)

sni_il = analysis_perDRG.GroupData(results_sni, 'IL', number_of_rats=6)
sni_cl = analysis_perDRG.GroupData(results_sni, 'CL', number_of_rats=6)

statistics_d7 = make_table(features, names)

In [6]:
name = 'd14perDRG_'

with open('D14_SNI_area.json') as f:
    results_sni = json.load(f)
with open('D14_Sham_area.json') as f:
    results_sham = json.load(f)
    
sham_il = analysis_perDRG.GroupData(results_sham, 'IL', number_of_rats=5)
sham_cl = analysis_perDRG.GroupData(results_sham, 'CL', number_of_rats=5)

sni_il = analysis_perDRG.GroupData(results_sni, 'IL', number_of_rats=6)
sni_cl = analysis_perDRG.GroupData(results_sni, 'CL', number_of_rats=6)

statistics_d14 = make_table(features, names)

In [8]:
with pd.ExcelWriter('Statistics_Bonferroni_adj.xlsx') as writer:  

    statistics_d7.to_excel(writer, sheet_name='d7')

    statistics_d14.to_excel(writer, sheet_name='d14')